In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn

In [84]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [85]:
dataset = pd.read_csv('Obfuscated-MalMem2022.csv')
x = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [86]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
ss=StandardScaler()
x=ss.fit_transform(x)
le=LabelEncoder()
y=le.fit_transform(y)

In [87]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state =42)

In [88]:
x_train=torch.FloatTensor(x_train).to(device)
x_test=torch.FloatTensor(x_test).to(device)
y_train=torch.FloatTensor(y_train).to(device)
y_test=torch.FloatTensor(y_test).to(device)

In [89]:
class BinClass(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1=nn.Linear(55,100,device=device)
        self.layer_2=nn.Linear(100,100,device=device)
        self.layer_3=nn.Linear(100,1,device=device)
        self.act=nn.ReLU()
        self.out=nn.Sigmoid()
    def forward(self,x):
        return self.out(self.layer_3(self.act(self.layer_2(self.act(self.layer_1(x))))))
    
model=BinClass().to(device)

In [90]:
loss_fn=nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.1)

In [91]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100 
    return acc

In [118]:
torch.manual_seed(42)
epochs=10001

for epoch in range(epochs):
    model.train()
    y_logits=model(x_train).squeeze()
    loss=loss_fn(y_logits,y_train)
    y_pred=torch.round(y_logits)
    acc=accuracy_fn(y_train,y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.inference_mode():
        test_logits=model(x_test).squeeze()
        test_loss=loss_fn(test_logits,y_test)
        test_pred=torch.round(test_logits)
        test_acc=accuracy_fn(y_test,test_pred)
        if(epoch%1000==0):
            print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.25873, Acc: 99.74% | Test Loss: 0.22184, Test Acc: 99.78%
Epoch: 1000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 2000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 3000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 4000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 5000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 6000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 7000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 8000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 9000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
Epoch: 10000 | Loss: 0.23466, Acc: 99.77% | Test Loss: 0.22544, Test Acc: 99.77%
